Importing all the required libraries
<hr>We'll use beautiful soup fot scrapping the data,tqdm to show the progress in a bar form,
Urllib for collecting several modules for working with URLs,



In [63]:
from time import sleep
from random import random
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import urllib.parse as urlparse
from urllib.parse import parse_qs

In [64]:

BASE_URL = 'https://www.flipkart.com/'
SEARCH_QUERY = "headphone"
TOP_N_PRODUCTS = 4
REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT = 10 

In [65]:
SAMPLE_URL = "https://www.flipkart.com/boat-rockerz-400-bluetooth-headset/product-reviews/itm14d0416b87d55?pid=ACCEJZXYKSG2T9GS&lid=LSTACCEJZXYKSG2T9GSVY4ZIC&marketplace=FLIPKART&page=1"
r = requests.get(SAMPLE_URL)    
soup = BeautifulSoup(r.content, 'html.parser') 
print(soup.prettify()[:500])

<!DOCTYPE doctype html>
<html lang="en">
 <head>
  <link href="https://rukminim1.flixcart.com" rel="preconnect"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app_modules.chunk.905c37.css" rel="stylesheet"/>
  <link href="//static-assets-web.flixcart.com/fk-p-linchpin-web/fk-cp-zion/css/app.chunk.104e9a.css" rel="stylesheet"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta 


In [75]:

## The class used in the html of flipkart review div is 'col _2wzgFH K0kLPL'


rows = soup.find_all('div',attrs={'class':'col _2wzgFH _16PB!m'})
# iteration over all blocks
for row in rows:
    # Print a sample row(review html block)
    # print(f"row:\n{row} \n\n")
    
    # finding all rows within the block
    sub_row = row.find_all('div',attrs={'class':'row'})
        
    # extracting text from 1st and 2nd row
    rating = sub_row[0].find('div').text
    print(f"rating:{rating} \n\n")
    
    summary = sub_row[0].find('p').text
    print(f"summary:{summary} \n\n")
    
    review = sub_row[1].find_all('div')[2].text
    print(f"review:{review} \n\n")
    
    location = sub_row[3].find('p',attrs={'class':'_2mcZGG'}).find_all('span')[1].text
    location = "".join(location.split(",")[1:]).strip()
    print(f"location:{location} \n\n")
    
    date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text
    print(f"date:{date} \n\n")
    
    
    sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})
    
    upvotes = sub_row_2[0].text
    print(f"upvotes:{upvotes} \n\n")
    
    downvotes = sub_row_2[1].text
    print(f"downvotes:{downvotes} \n\n")
    
    break

Search based Product URL 

In [67]:
def get_popular_product_s_titles_and_urls(search_query : str, popular_products_count_limit : int = None):
    
    search_url = f"{BASE_URL}search?q={search_query}&sort=popularity"
    search_response = requests.get(search_url)
    
    # Pause the loop for 1-3 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
    # sleep(randint(1,3))
    
    search_html_soup = BeautifulSoup(search_response.content, 'html.parser')
    search_results_products = search_html_soup.find_all('div',attrs={'class':'_4ddWXP'})
    
    product_titles, product_urls = [],[]
    
    product_count = 0
    
    for product in tqdm(search_results_products, desc="Search Results Iteration", position=0, leave=True):
        
        ad_mention_subrow = product.find("div", attrs={"class":"_4HTuuX"})
        
        is_ad = not not ad_mention_subrow
        
        if not is_ad:
            
            title_mention_subrow = product.find("a", attrs={"class":"s1Q9rs"})
            
            product_title = title_mention_subrow["title"]
            product_relative_url = title_mention_subrow["href"]
            product_url = urljoin(BASE_URL,product_relative_url)
            
            parsed_url = urlparse.urlparse(product_url)
            parsed_url_path = parsed_url.path
            parsed_url_path_split = parsed_url_path.split("/")
            parsed_url_path_split[2] = "product-reviews"
            parsed_url_path_modified = "/".join(parsed_url_path_split)
            parsed_url_modified = parsed_url._replace(path=parsed_url_path_modified)
            product_url = parsed_url_modified.geturl()
            
            product_titles.append(product_title)
            product_urls.append(product_url)
            
            product_count += 1
            
            if popular_products_count_limit and (product_count >= popular_products_count_limit):
                break
                
    return product_titles, product_urls

In [68]:
product_titles, product_urls = get_popular_product_s_titles_and_urls(SEARCH_QUERY, TOP_N_PRODUCTS);

Search Results Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

# Iterating over multiple products and multiple pages

In [69]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["# Products", "# Reviews Per Page", "# Pages", "# Total Reviews Count"]
x.add_row([len(product_urls), 10, REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT, len(product_urls)*10*REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT])
print(x)

+------------+--------------------+---------+-----------------------+
| # Products | # Reviews Per Page | # Pages | # Total Reviews Count |
+------------+--------------------+---------+-----------------------+
|     4      |         10         |    10   |          400          |
+------------+--------------------+---------+-----------------------+


In [70]:
dataset = []

for idx, url in enumerate(tqdm(product_urls, desc='products')):
    # iterating over review pages
    for i in tqdm(range(1,REVIEW_PAGES_TO_SCRAPE_FROM_PER_PRODUCT+1), desc="review pages", position=0, leave=False):
        parsed = urlparse.urlparse(url)
        pid = parse_qs(parsed.query)['pid'][0]
        URL = f"{url}&page={i}"
        
        r = requests.get(URL)
        
        # Pause the loop for 0-1 seconds to simulate natural setting not overwhelm the server with back to back requests without any pause
        sleep(random())
        soup = BeautifulSoup(r.content, 'html.parser') 

        rows = soup.find_all('div',attrs={'class':'col _2wzgFH K0kLPL'})

        for row in rows:

            # finding all rows within the block
            sub_row = row.find_all('div',attrs={'class':'row'})

            # extracting text from 1st 2nd and 4th row
            rating = sub_row[0].find('div').text
            summary = sub_row[0].find('p').text
            summary = summary.strip()
            review = sub_row[1].find_all('div')[2].text
            review = review.strip()
            location=""
            location_row = sub_row[3].find('p',attrs={'class':'_2mcZGG'})
            if location_row:
                location_row = location_row.find_all('span')
                if len(location_row)>=2:
                    location = location_row[1].text
                    location = "".join(location.split(",")[1:]).strip()
            date = sub_row[3].find_all('p',attrs={'class':'_2sc7ZR'})[1].text

            sub_row_2 = row.find_all('div',attrs={'class':'_1e9_Zu'})[0].find_all('span',attrs={'class':'_3c3Px5'})

            upvotes = sub_row_2[0].text
            downvotes = sub_row_2[1].text

            # appending to data
            dataset.append({'product_id':pid, 'product_title':product_titles[idx], 'rating': rating, 'summary': summary, 'review': review, 'location' : location, 'date' : date, 'upvotes' : upvotes, 'downvotes' : downvotes})

products:   0%|          | 0/4 [00:00<?, ?it/s]

review pages:   0%|          | 0/10 [00:00<?, ?it/s]

review pages:   0%|          | 0/10 [00:00<?, ?it/s]

review pages:   0%|          | 0/10 [00:00<?, ?it/s]

review pages:   0%|          | 0/10 [00:00<?, ?it/s]

View Sample set of reviews that we collected

In [74]:
df = pd.DataFrame(dataset)

with pd.option_context('display.max_colwidth', -1):
    display(df.head(10))
    display(df.tail(10))

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
0,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Great product,My one request🙏 plz to all buyers..plz dont review any product specially this.. on that day when it delivered to your homeplzz review after 10-15 days..I'm sure your 3-4 star will convert into 5 star..bcoz in first day it will look bulky nd big but after using some days it will look light weight nd normal size nd also its sound quality will seem more awsm..!!I m in love with my rockerz 510.. i'm writing this after 15 days usage nd now my ears are not paining even 2-3 hours continue usage.....,Jabalpur,"Nov, 2018",8166,959
1,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Classy product,genuine review1. sound quality : no issue value for money2. build quality : plastic best 3. look: very cool 4. pain: used 6to7 hr per day no pain (online classes)5. call: very comfortable like u r with phone6.connectivity: faadu hai.. excellent7.overall: blindly go for it,Noida,"Oct, 2018",4904,612
2,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Mind-blowing purchase,Nice product. Happy with the bass quality. One information I would like to share that the logo has been changed so don't need to worry.I hope this review will help you.,Kanpur,"May, 2019",2341,323
3,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Highly recommended,"Pros:1. Connectivity with bleutooth and aux.2. Noise Cancellation.3. Cushioning is great.4. Bass is best in class. No other headphone can beat it's bass. I also use Sony headphones of same price point but these headphones have better durability.5. Seller and Flipkart did there work effectively with early delivery than the promised date.6. Amazing battery life.7. Overall sound quality is good with responsive low,mid and high.8. Can play different Geners effectively.Cons:1. Not Comfor...",Udaipur,"Jul, 2019",393,52
4,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Simply awesome,"super sound quality, and battery backup is awesome built quality also excellent aux cable also provided so it works with out battery.",Warangal,"Nov, 2018",972,155
5,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,4,Very Good,Sound of awesome 👌👌👌👌👌Base is good 👍👍👍👍👍 build design good 💪💪💪💪💪Battery life super 🤙🤙🤙🤙🤙Call quality excellent 🤞🤞🤞🤞🤞worth of money 👏👏👏👏👏,Chennai,"Feb, 2020",581,89
6,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Excellent,"Trimendus bass with 50mm driver... awesome trendy design, good bluetooth coverage, nice battery backup... I love it...",Jagadanandapur,"May, 2019",683,116
7,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,4,Good choice,"Good for :- PUBG HEADPHONE ( BUY IT if gamer HD footsteps ), i cant say lag in wirless play..its okay i never felt lag. Aux can be used if as well.- comforts . No ear pain for hours- Music- movies- Long battery backup lasts a week and more.Not that good:- Overpriced- BoAt could have improved the sound @this price- Portabily , can't fold or carry it safely. Not Recommended for those who like to travel and carry it with. Overall:Me :-Bought for Pubg gaming,watch movies,long lastin...",Oachira,"May, 2020",611,115
8,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Awesome,Super bass sound quality very nice very high sound good product battery backup UPTU 12 hours playback non stop value for money nice product,New Delhi,"Sep, 2018",462,96
9,ACCEZB5HSGRXTYVF,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Terrific purchase,"Boat 510 is awesomesound is super clear, also on full volume.bass is very very good like thumping,battery backup is also super it's still running from a week,design is hot I mean it's so smooth and shiny.overall full value of money.if you planning to buy a headphone no doubt go for it, without reading any review.cons-nothing sti

,product_id,product_title,rating,summary,review,location,date,upvotes,downvotes
380,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,3,Just okay,I think it has average quality sound but base is Good in this price range mot important feature is i use it when battery is low and it work properly battery life is good,Kanpur,"Aug, 2020",9,5
381,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Terrific purchase,amaizng bass battery backup very good design superb and gym lover also love it,New Delhi,"Oct, 2019",9,5
382,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Terrific purchase,boAt ROCKERZ 510 is very good productReview after 15 -20 daysBattery - very very .... good (10/10)Design - i think design wise i will give (9/10)Quality - great(10/10)Connectivity - very very .......fast(10/10)overall view product is amazing 👌,Raghogarh,"Jan, 2020",4,1
383,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Super!,"Great BT Headsets for Bass Heads !! Battery life is good, built quality is also nice. Give it a time for 3-4 days to open up. Not for people looking for crisp sound. Good for bass type music and general video watching. Overall it is Value For Money product. Go for it.",Nashik,"Oct, 2019",4,1
384,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Super!,"Battery Backup is awesome! It works without charging for couple of weeks on using 1-2 hours daily. Sound Quality is awesome and its same as compared to Mi wired Headphones, may be little but more. Design is awesome but wearing it more than 30-45 minutes it starts to hurt your ears. Cusion could have been much softer. Overall, its best out there and recommended to anyone.",Thane,"Sep, 2019",4,0
385,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Perfect product!,I am highly impressed with this product. It is best in every aspects specially for wfh and online classes. Anyone can talk on call easily. I am highly impressed with delivery experience specially open box delivery.,Jabalpur,1 month ago,1,0
386,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,3,Fair,Purchased on 6 October 2021 .. honest review on 20 September... Plastic build is solid and not much flexible not foldable too ND too much treble no bass is there. u can't listen to anything at full volume.... headache.... cushioning are OK OK....when any bass comes it started to make abnormal noises.....,Gwalior,1 month ago,1,0
387,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Brilliant,"Fits properly n covers full ears, seems no heavy after using for too long.Battery backup is really appreciable n sound wise perfect.Can go for it 😊",Faridabad,3 months ago,1,0
388,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,4,Very Good,"this review will be helpful for those who have used sony xb55ap, i have used xb for 3 years, so my review will be based on it1. For Music :- Its good, but not anywhere when compared to xb 55 ap2. For Gaming :- when using with cable there is no latency, but when connected with bluetooth there is considerable latency, Also audio quality in game is not upto the mark. Footsteps of enemies is not audible from a distance, but with xb it is better3. Long time usage:- not for long time use as it ...",Visakhapatnam,4 months ago,2,0
389,ACCF5GXGMMVPDQVD,boAt Rockerz 510 Super Extra Bass Bluetooth Headset,5,Excellent,The product is very good the noise cancellation is not that much good but I think u can still try it and most importantly u might write ther review after 10 to 15 days of usage I am also writing it ater 7 days and there is no problem till now the bass quality is very good and the design is badass in last it is worth to buy it Thank You For Reading 😊😊 (This is worth to buy u can use for daily use and gaming)🙏,Faridabad,"Mar, 2021",2,0


In [72]:
count_reviews = df.shape[0]
print(f"Count of reviews:{count_reviews}")

Count of reviews:390


saving the scrapped data to a csv file

In [73]:
df.to_csv("./flipkart_reviews_dataset.csv", index=False)